In [269]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [270]:
data = pd.read_csv('../datasets/AmesHousing.tsv', delimiter="\t")

In [271]:
def transform_features(data):
    return df

In [272]:
def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]

In [273]:
def train_and_test(df):
    train = df.iloc[:1460]
    test = df.iloc[1460:]
    
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error
    
    features = train.select_dtypes(include=['int', 'float']).drop(columns="SalePrice").columns
    lr = LinearRegression()
    lr.fit(train[features], train['SalePrice'])

    test_predictions = lr.predict(test[features])

    test_mse = mean_squared_error(test_predictions, test['SalePrice'])
    
    test_rmse = np.sqrt(test_mse)

    return test_rmse

filtered_df = select_features(data)
rmse = train_and_test(filtered_df)
rmse

57088.25161263909

# Clean the data and select transformations

In [274]:
#1 drop columns with more than 25% missing values
missing_val = data.isnull().sum()
drop_cols = missing_val[(missing_val/len(data)) > 0.05]
data = data.drop(columns=drop_cols.index)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 71 columns):
Order              2930 non-null int64
PID                2930 non-null int64
MS SubClass        2930 non-null int64
MS Zoning          2930 non-null object
Lot Area           2930 non-null int64
Street             2930 non-null object
Lot Shape          2930 non-null object
Land Contour       2930 non-null object
Utilities          2930 non-null object
Lot Config         2930 non-null object
Land Slope         2930 non-null object
Neighborhood       2930 non-null object
Condition 1        2930 non-null object
Condition 2        2930 non-null object
Bldg Type          2930 non-null object
House Style        2930 non-null object
Overall Qual       2930 non-null int64
Overall Cond       2930 non-null int64
Year Built         2930 non-null int64
Year Remod/Add     2930 non-null int64
Roof Style         2930 non-null object
Roof Matl          2930 non-null object
Exterior 1st       29

In [275]:
#2 drop text cols with more than 1 missing val
text_cols = data.select_dtypes(include=["object"]).isnull().sum()
text_cols_drop = text_cols[text_cols > 0]
data = data.drop(columns=text_cols_drop.index)
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 64 columns):
Order              2930 non-null int64
PID                2930 non-null int64
MS SubClass        2930 non-null int64
MS Zoning          2930 non-null object
Lot Area           2930 non-null int64
Street             2930 non-null object
Lot Shape          2930 non-null object
Land Contour       2930 non-null object
Utilities          2930 non-null object
Lot Config         2930 non-null object
Land Slope         2930 non-null object
Neighborhood       2930 non-null object
Condition 1        2930 non-null object
Condition 2        2930 non-null object
Bldg Type          2930 non-null object
House Style        2930 non-null object
Overall Qual       2930 non-null int64
Overall Cond       2930 non-null int64
Year Built         2930 non-null int64
Year Remod/Add     2930 non-null int64
Roof Style         2930 non-null object
Roof Matl          2930 non-null object
Exterior 1st       29

In [276]:
#3 fill in numerical columns with less than 5% missing values with the mode
numerical_cols = data.select_dtypes(include=['int', 'float'])
num_missing_val = data.isnull().sum()
mode_replace_cols = num_missing_val[((num_missing_val/len(data)) < 0.05) &((num_missing_val/len(data)) > 0)] 
for col in mode_replace_cols.index:
    data[col].fillna(int(data[col].mode()), inplace=True)
data.isnull().sum().value_counts()

0    64
dtype: int64

In [284]:
#4 new features to built: 
# a) how old has it been at sale YEARS_sold
# b) how long has last remod been ago years_remod

data["years_sold"] = data["Yr Sold"] - data["Year Built"]
data["years_remod"] = data["Yr Sold"] - data["Year Remod/Add"]

data = data.drop(data[data["years_sold"] < 0].index)
data = data.drop(data[data["years_remod"] < 0].index)

In [285]:
data["years_sold"].value_counts().sort_index()

0      114
1      218
2       90
3       60
4       76
5       66
6       66
7       51
8       44
9       50
10      40
11      47
12      33
13      38
14      27
15      31
16      22
17      25
18      14
19      16
20       9
21       9
22      12
23      12
24       8
25      11
26      17
27       9
28      19
29      30
      ... 
97      11
98      10
99      13
100      5
101      1
102      1
103      3
104      1
106     10
107      5
108      7
109      5
110      5
111      1
112      2
113      1
114      2
115      1
117      3
118      2
119      2
120      1
122      1
125      1
126      1
127      2
128      2
129      2
135      1
136      1
Name: years_sold, Length: 127, dtype: int64

In [286]:
data["years_remod"].value_counts().sort_index()

0     238
1     193
2     104
3      97
4     111
5      91
6      83
7      75
8      85
9      71
10     66
11     62
12     59
13     57
14     46
15     37
16     38
17     33
18     18
19     21
20     18
21     15
22     13
23     18
24     10
25     14
26     17
27     18
28     22
29     26
     ... 
31     50
32     37
33     29
34     30
35     22
36     37
37     30
38     27
39     47
40     30
41     26
42     33
43     27
44     34
45     22
46     25
47     28
48     26
49     27
50     23
51     31
52     29
53     30
54     21
55     23
56    112
57     79
58     76
59     78
60     42
Name: years_remod, Length: 61, dtype: int64

In [288]:
## Drop columns that aren't useful for ML
data = data.drop(["PID", "Order"], axis=1)

## Drop columns that leak info about the final sale
data = data.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

In [290]:
def transform_features(data):
    #1 drop columns with more than 25% missing values
    missing_val = data.isnull().sum()
    drop_cols = missing_val[(missing_val/len(data)) > 0.05]
    data = data.drop(columns=drop_cols.index)
    
    #2 drop text cols with more than 1 missing val
    text_cols = data.select_dtypes(include=["object"]).isnull().sum()
    text_cols_drop = text_cols[text_cols > 0]
    data = data.drop(columns=text_cols_drop.index)

    #3 fill in numerical columns with less than 5% missing values with the mode
    numerical_cols = data.select_dtypes(include=['int', 'float'])
    num_missing_val = data.isnull().sum()
    mode_replace_cols = num_missing_val[((num_missing_val/len(data)) < 0.05) &((num_missing_val/len(data)) > 0)] 
    for col in mode_replace_cols.index:
        data[col].fillna(int(data[col].mode()), inplace=True)
        
    #4 new features to built: 
    # a) how old has it been at sale YEARS_sold
    # b) how long has last remod been ago years_remod

    data["years_sold"] = data["Yr Sold"] - data["Year Built"]
    data["years_remod"] = data["Yr Sold"] - data["Year Remod/Add"]

    data = data.drop(data[data["years_sold"] < 0].index)
    data = data.drop(data[data["years_remod"] < 0].index)
    
    ## Drop columns that aren't useful for ML
    data = data.drop(["PID", "Order"], axis=1)

    ## Drop columns that leak info about the final sale
    data = data.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)
    return data

data = pd.read_csv('../datasets/AmesHousing.tsv', delimiter="\t")
transform_df = transform_features(data)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

55275.36731241307

In [292]:
import seaborn as sns

ImportError: No module named 'seaborn'